In [1]:
import derinet.lexicon as dlex
import os
lexicon = dlex.Lexicon()
current_dir = os.getcwd()  # aktualni adresar
file_path = os.path.join(current_dir, "./derinet-2-3.tsv")  #sestaveni cesty
lexicon.load(file_path)

testování 1.1: vyhledávání slov, které jsou stejná, ale jsou tam vícekrát (Adam, Adam)
    - uvést druh daného slova a jestli je osamocen nebo má nějakého rodiče (absurdno, které tam je dvakrát, jednou o samotě, jednou ve stromě)
testování 1.2: vyhledávání slov, které jsou stejná, liší se pouze tím, že je na začátku jiné písmeno velikostně (absurdistán, Absurdistán)
    - to samé

In [11]:
#testování 1
videno = set()
with open("test1.1.txt", "w", encoding="utf-8") as f:
    for lexeme in lexicon.iter_lexemes():
        if lexeme.lemma not in videno:
            dalsi = lexicon.get_lexemes(lemma=lexeme.lemma)
            if len(dalsi) > 1:
                dalsi.pop(0)
                for lex in dalsi:
                    videno.add(lexeme.lemma)
                    if lexeme.pos != lex.pos: #pokud mají různé typy
                        f.write(f"{lexeme.lemma}, {lex.lemma} -  RŮZNÉ typy: {lexeme.pos}, {lex.pos} \n") #nejdříve původní, potom nalezlý
                    else:
                        f.write(f"{lexeme.lemma}, {lex.lemma} - STEJNÝ typ: {lex.pos} \n")

videno.clear()

with open("test1.2.txt", "w", encoding="utf-8") as f:
            #ted 1.2 - různá velikost začínajících písmen
    for lexeme in lexicon.iter_lexemes():
        if lexeme.lemma not in videno and not lexeme.lemma.startswith("-"):
            cast = lexeme.lemma[1:]
            pismeno = lexeme.lemma[0:1].swapcase()
            lexeme1 = pismeno + cast #přetvořím si to slovo tak, aby tady bylo na začátku obrácená velikost písmena
            for lex in lexicon.get_lexemes(lemma=lexeme1): #najdu jestli, tam je nějaké obrácené velikosti
                videno.add(lexeme.lemma)
                videno.add(lexeme1)
                #oboje tam přidám
                if lexeme.pos != lex.pos: #pokud mají různé typy
                    f.write(f"!!! {lexeme.lemma}, {lex.lemma} -  RŮZNÉ typy: {lexeme.pos}, {lex.pos} \n") #nejdříve původní, potom nalezlý
                else:
                    if lexeme.pos == "ADJ" or lexeme.pos == "NOUN":
                        f.write(f"{lexeme.lemma}, {lex.lemma} - STEJNÝ typ: {lex.pos} \n")
                    else:
                        f.write(f"!!! {lexeme.lemma}, {lex.lemma} - STEJNÝ typ, co není ADJ nebo NOUN: {lex.pos} \n")


In [147]:
prirazeni = {
    "jedna" : "jeden",
    "dvoust" : "dvěstě",
    "desát" : "deset",
    "devát" : "devět",
    "třicát" : "třicet",
    "čtyřicát" : "čtyřicet",
    "dvacát" : "dvacet",
    "prv" : "prvé",
    "pát" : "pět",
    "pětist" : "pětset",
    "sedmist" : "sedmset", 
    "st" : "sto",
    "osedm" : "sedm",
    "pátnáct" : "patnáct",
    "šedesáté" : "šedesát",
    "druh" : "dvě"
}

In [108]:
for lexeme in lexicon.iter_lexemes():
    if lexeme.lemma == "stotřicet":
        slovo = lexeme.children
        print(f"{slovo}")
        break

[]


In [31]:

def je_samohlaska(znak):
    return znak in "aeiouyáéěíóúůý"

def deleni_slova(slovo):
    souhlaska_na_konci = ["jeden", "pět", "šest", "sedm", "osm", "devět"]
    vyjimky = ["sto", "jedna", "šedesáté"]
    casti = list()
    cteme = True
    pomoc = ""

    for i in range(len(slovo)): 
        if i == len(slovo) - 1:
            casti.append(pomoc)

        elif cteme and slovo[i] != "a":
            if pomoc in vyjimky:
                casti.append(pomoc)
                pomoc = slovo[i]
            else:
                pomoc += slovo[i]
                cteme = not je_samohlaska(slovo[i])
        elif cteme and slovo[i] == "a":
            if pomoc in souhlaska_na_konci:
                casti.append(pomoc)
                pomoc = ""
            else:
                pomoc += "a"
                cteme = False 
        
        elif not cteme and not slovo[i] == "a":
            if pomoc in vyjimky:
                casti.append(pomoc)
                pomoc = slovo[i]
                cteme = True
            else:
                pomoc += slovo[i]
                cteme = not je_samohlaska(slovo[i])
        elif not cteme and slovo[i] == "a":
            casti.append(pomoc)
            pomoc = ""
            cteme = True
            
    return casti

casti = deleni_slova("jednadvacáté")
for i in casti: 
    print(i)

jedna
dvacát


testování 2: přídavná jména končící na -ní by měla být vždy připojena k nějakému slovu
    - využívám v misc toho, že je tam unmotivated, tedy není odvozeno od jiného slova

In [51]:
with open("test2.txt", "w", encoding="utf-8") as f: #kdyz tam dodam with, tak se mi to samo zavre
    for lexeme in lexicon.iter_lexemes():
        if lexeme.pos == "ADJ" and lexeme.lemma.endswith("ní"):
            if "unmotivated" not in lexeme.misc: #tedy nejsou odvozena od jiného českého slova
                rodic = lexeme.all_parents 
                if len(rodic) == 0:
                    f.write(f"{lexeme.lemma} \n")

#potřebuju rozlišit, když to budou slova přejatá z jiného jazyka - lumbální tu nemá co dělat, ale fauní by mělo být u fauna nebo faun (??)

testování 3: číslovková příslovce - udělat slovník na číslovky a přiřazovat je k nim 
    - tohle by se potom dalo udělat i pro přídavná čísla??

In [ ]:
def je_samohlaska(znak):
    return znak in "aeiouyáéěíóúůý"

def deleni_slova(slovo):
    souhlaska_na_konci = ["jeden", "pět", "šest", "sedm", "osm", "devět"]
    vyjimky = ["sto", "jedna"]
    casti = list()
    cteme = True
    pomoc = ""

    for i in range(len(slovo)): 
        if i == len(slovo) - 1:
            casti.append(pomoc)

        elif cteme and slovo[i] != "a":
            if pomoc in vyjimky:
                casti.append(pomoc)
                pomoc = slovo[i]
            else:
                pomoc += slovo[i]
                cteme = not je_samohlaska(slovo[i])
        elif cteme and slovo[i] == "a":
            if pomoc in souhlaska_na_konci:
                casti.append(pomoc)
                pomoc = ""
            else:
                pomoc += "a"
                cteme = False 
        
        elif not cteme and not slovo[i] == "a":
            if pomoc in vyjimky:
                casti.append(pomoc)
                pomoc = slovo[i]
                cteme = True
            else:
                pomoc += slovo[i]
                cteme = not je_samohlaska(slovo[i])
        elif not cteme and slovo[i] == "a":
            casti.append(pomoc)
            pomoc = ""
            cteme = True
            
    return casti

def hledani(casti):
    konec = []
    nezarazene = []

    # nejdřív přiřadíme z prirazeni
    for cast in casti:
        if cast in prirazeni:
            konec.append(prirazeni[cast])
        else:
            nezarazene.append(cast)

    # pak zkoušíme hledat v lexikonu
    casti = []
    for cast in nezarazene:
        nalezeno = False
        for lex in lexicon.iter_lexemes():
            if lex.lemma == cast:
                konec.append(cast)
                nalezeno = True
                break
        if not nalezeno:
            casti.append(cast)

    f.write(f"{lexeme.lemma} má nejspíš tyto rodiče:")
    for i in konec:
        f.write(f"{i},")
    f.write("\n")

    if casti:
        f.write(f"!!! NEZAŘAZENÉ ČÁSTI:")
        for i in casti:
            f.write(f"{i}")
        f.write("\n")

with open("test3.txt", "w", encoding="utf-8") as f: #kdyz tam dodam with, tak se mi to samo zavre
    for lexeme in lexicon.iter_lexemes():
        if lexeme.pos == "ADV" and lexeme.parent is None and "unmotivated" not in lexeme.misc and lexeme.lemma.endswith("é"):
            #a teď mám několik verzí, co by to mohlo být 
            if lexeme.lemma.startswith("napo"):
                slovo = lexeme.lemma[4:]
            elif lexeme.lemma.startswith("po"):
                slovo = lexeme.lemma[2:]
            elif lexeme.lemma.startswith("za"):
                slovo = lexeme.lemma[2:]
            elif lexeme.lemma.startswith("na"):
                slovo = lexeme.lemma[2:]
            elif lexeme.lemma.startswith("propo"):
                slovo = lexeme.lemma[5:]
            else:
                slovo = lexeme.lemma

            casti = deleni_slova(slovo)
            hledani(casti)

testování 4: přídavná jména končící na 'istický' by měla být připojena k 'ismus' podstatné jméno

In [136]:
with open("test4.txt", "w", encoding="utf-8") as f:
    for lexeme in lexicon.iter_lexemes():
        if lexeme.lemma.endswith("istický") and lexeme.parent is None:
            if not lexeme.lemma.endswith("statistický") and not lexeme.lemma.startswith("proti"):
                nove = lexeme.lemma[:-6]
                nove = nove + "smus"
                existuje = False
                for lex in lexicon.iter_lexemes():
                    if lex.lemma == nove:
                        f.write(f"{lexeme.lemma}, {lex.lemma} \n")
                        existuje = True
                    if existuje:
                        break
                if not existuje:
                    f.write(f"!!! {lexeme.lemma}, ale neexistuje {nove} \n")

            if lexeme.lemma.endswith("statistický"):
                f.write(f"STATISTICKÝ: {lexeme.lemma} byl měl být pod statistikou \n")
            
            if lexeme.lemma.startswith("proti"):
    
                nove = lexeme.lemma[:-6][5:]
                nove = nove + "smus"
                f.write(f"PROTI: {lexeme.lemma}, {nove} \n")



testování 5: slova, které nejsou k ničemu připojena, ale jsou podslovem jiných (adamantin - adaamantinoma)
    - tenhle test musím nechat doběhnout až budu vařit

In [ ]:
def lezeme_nahoru(dite, hledany):
    rodice = dite.all_parents
    if not rodice:
        return False  # nedošli jsme k hledanému
    if hledany in rodice:
        return True  # hledaný je přímo rodičem
    for r in rodice:
        if lezeme_nahoru(r, hledany):  # rekurzivně jdeme dál
            return True
    return False  # pokud jsme ho nikdy nenašli


i = 0
with open("test5.txt", "w", encoding="utf-8") as f:
    for lexeme in lexicon.iter_lexemes():
            for lex in lexicon.iter_lexemes():
                if len(lexeme.lemma) < len(lex.lemma) and lexeme.lemma in lex.lemma:
                    dobry = True
                    for i in lex.all_parents:
                        if i != lexeme.lemma:
                            predek = lezeme_nahoru(lex, lexeme)
                            if predek:
                                dobry = False
                    if dobry :
                        print(f"{lex.lemma} vypadá, že je od {lexeme.lemma}")
                        i += 1
                        if i == 50:
                            break

        

TypeError: 'list' object is not callable

testování 6: když to končí na 'čin' a 'ův', tak to musí být napojeno na mužské a ženské podstatné jméno 
    - 6.1 - ženský rod

In [ ]:
with open("test6.1.txt", "w", encoding="utf-8") as f:
    for lexeme in lexicon.iter_lexemes():
        if lexeme.lemma.endswith("čin") and lexeme.pos == "ADJ":
            #tady je to dobře, protože to má být hned nad dítětem
            rodice = lexeme.all_parents
            jeden = False # jestli jsme našli nějakou předkyni
            jiny = False # pokud jsme ale nasli predka, co neni zensky, ale sedí do popisu
            slovo = None
            for r in rodice:
                if r.lemma[:-2] == lexeme.lemma[:-3]:
                    jiny = True 
                    if r.pos == "NOUN" and r.feats["Gender"] == "Fem":
                        jeden = True
                    else: 
                        slovo = r.lemma

                
                
            if not jeden and jiny:
                f.write(f"PŘEDEK NENÍ ŽENSKÉHO RODU: {lexeme.lemma} a {slovo} \n")
            elif not jeden and not jiny:
                volny = True
                for lex in lexicon.iter_lexemes():
                    if lex.lemma[:-2] == lexeme.lemma[:-3] and lex.lemma != lexeme.lemma and lex.lemma.endswith("a"):
                        if lex.pos == "NOUN" and lex.feats["Gender"] == "Fem":
                            f.write(f"NEJSOU K SOBĚ PŘIPOJENI: {lexeme.lemma} a {lex.lemma} \n")
                        else:
                            f.write(f"NEJSOU K SOBĚ PŘIPOJENI A NESEDÍ ROD PŘEDKA: {lexeme.lemma} a {lex.lemma} \n")
                        volny = False
                        break
                if volny: 
                    f.write(f"!!!! BEZ PŘEDKA {lexeme.lemma} \n")
                    

testování 6.2: teď pro muže 

In [ ]:
def tvorba_predka(slovo):
    temp = slovo[:-2]
    nove = temp[0:-1]
    nove2 = temp[-1]
    slovicko = nove + "e" + nove2
    return slovicko
    
def hledani_predka(lexeme):
    rodice = lexeme.all_parents
    pomocnik = tvorba_predka(lexeme.lemma)
    jeden = False # jestli jsme našli nějakou předkyni
    jiny = False # pokud jsme ale nasli predka, co neni muzsky, ale sedí do popisu
    slovo = None
    for r in rodice:
        re = r.lemma
        if r.lemma[-1] in "aeiouyáéíóúůý" and r.lemma[-2] not in "aeiouyáéíóúůý":
            re = r.lemma[:-1]
        if re.endswith("os"):
            re = r.lemma[:-2]
        if r and (re == pomocnik or re == lexeme.lemma[:-2]):
            jiny = True 
            if r.pos == "NOUN" and "Gender" in r.feats.keys() and r.feats["Gender"] == "Masc":
                jeden = True
                break
            else: 
                slovo = r.lemma

    return jeden, jiny, slovo

with open("test6.2.txt", "w", encoding="utf-8") as f:
    for lexeme in lexicon.iter_lexemes():
        if lexeme.lemma.endswith("ův") and lexeme.pos == "ADJ": #and not test_samohlasek(lexeme.lemma[:-3]):
            #tady je to dobře, protože to má být hned nad dítětem
            #teď mám posichrováno, pokud je to tvaru 'aristokrat > aristokratův'
            jeden, jiny, slovo = hledani_predka(lexeme)
        
            if not jeden and jiny:
                f.write(f"PŘEDEK NENÍ MUŽSKÉHO RODU: {lexeme.lemma} a {slovo} \n")
            elif not jeden and not jiny:
                volny = True
                pomocnik = tvorba_predka(lexeme.lemma)
                for lex in lexicon.iter_lexemes():
                    if lex.lemma == lexeme.lemma[:-2] or lex.lemma == pomocnik:
                        if lex.feats["Gender"] == "Masc":
                            f.write(f"NEJSOU K SOBĚ PŘIPOJENI: {lexeme.lemma} a {lex.lemma} \n")
                        else:
                            f.write(f"NEJSOU K SOBĚ PŘIPOJENI A NESEDÍ ROD PŘEDKA: {lexeme.lemma} a {lex.lemma} \n")
                        volny = False
                        break
                if volny: 
                    f.write(f"!!!! BEZ PŘEDKA {lexeme.lemma} \n")


testování 7: tvary jako 'čtyřka' má být u čtyři a podobně

In [ ]:
prevod = {
    "jednička" : "jedna",
    "dvojka" : "dva", 
    "trojka" : "tři",
    "čtyřka": "čtyři",
    "pětka": "pět",
    "šestka" : "šest",
    "sedmička" : "sedm",
    "osmička" : "osm",
    "devítka" : "devět", 
    "desítka" : "deset"
}

In [100]:
with open("test7.txt", "w", encoding="utf-8") as f:
    for lexeme in lexicon.iter_lexemes():
        if lexeme.lemma in prevod.keys(): #tak je tam prostě to slovo (není to dvacetjednička, ale jenom jednička)
            for k in prevod.keys(): 
                if k == lexeme.lemma: 
                    rodice = lexeme.all_parents
                    if prevod[k] not in rodice: 
                        f.write(f"MĚLI BY BÝT SPOJENI: {prevod[k]} a {lexeme.lemma} \n")

        elif lexeme.lemma.endswith("náctka"): 
            rodic = lexeme.lemma[:-2]
            for i in lexicon.iter_lexemes():
                if i.lemma == rodic and i not in lexeme.all_parents:
                    f.write(f"MĚLI BY BÝT SPOJENI: {rodic} a {lexeme.lemma} \n")
        
        elif lexeme.lemma.endswith("ka") and not lexeme.lemma.endswith("strojka"):
            for k in prevod.keys(): 
                if k in lexeme.lemma:
                    cast = lexeme.lemma.removesuffix(k)
                    rodice = lexeme.all_parents
                    if prevod[k] not in rodice: 
                        f.write(f"MĚLI BY BÝT SPOJENI: {prevod[k]} a {lexeme.lemma} \n")

                    if cast != "super":
                        for lex in lexicon.iter_lexemes():
                            #chci, aby pro stošestka se mi jako rodič našlo stošest a ne sto a šest
                            #tak ne, je to rozděleně -> sto šest
                            if lex.lemma == cast and cast not in rodice:
                                f.write(f"MĚLI BY BÝT SPOJENI: {cast} a {lexeme.lemma} \n")
                                break